Encoder

- data 파일 속 초록 값의 embedding 을 계산해, 새로운 열로 추가한 output 파일을 추출





In [ ]:
pip install -U sentence-transformers

In [ ]:
# 마운트
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import os
import numpy as np

In [ ]:
# 평균 풀링 - 주의 마스크(attention_mask)를 올바르게 고려하여 평균 계산
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # model_output의 첫 번째 요소에는 모든 토큰 임베딩이 포함됩니다
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
# 학습된 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')


In [ ]:
# 경로 지정 및 타겟값 지정
csv_file_path = '/content/drive/MyDrive/23-2 Industrial Engineering/Final_data.csv'  # Update the path as needed
target_column = '초록'  # Replace with the actual column name

In [ ]:
#output_directory = '/content/drive/MyDrive/23-2 Industrial Engineering/output'
#os.makedirs(output_directory, exist_ok=True)

# 데이터 로드
df = pd.read_csv(csv_file_path)

# 인덱스 섞기
shuffled_indices = np.random.permutation(df.index)
df_shuffled = df.iloc[shuffled_indices]

# 데이터프레임 600 분할
num_partitions = 600
partition_size = len(df) // num_partitions
df_partitions = [df_shuffled[i:i+partition_size] for i in range(0, len(df_shuffled), partition_size)]

# 각 데이터프레임을 저장
for i, df_partition in enumerate(df_partitions):
    globals()[f'df_{i+1}'] = df_partition


# 추출 경로 지정
sampled_csv_path = '/content/drive/MyDrive/23-2 Industrial Engineering/output_total.csv'  # Update with the desired path




In [ ]:
# 데이터 수합 프레임 생성
merged_df = pd.DataFrame()

# 반복
for i in range(51, 81):
    # 임배딩 값 계산
    sample_sentences = list(globals()[f'df_{i}'][target_column])
    encoded_inputs = tokenizer(sample_sentences, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_inputs)
    sample_embeddings = mean_pooling(model_output, encoded_inputs['attention_mask'])

    # 임베딩 값을 NumPy array 로 변환
    sample_embeddings_array = sample_embeddings.numpy()

    # 새로운 열로 추가
    globals()[f'df_{i}']['embeddings'] = sample_embeddings_array.tolist()  # You can rename 'embeddings' to the desired column name

    # 데이터 수합 프레임에 추가
    merged_df = pd.concat([merged_df, globals()[f'df_{i}']])

# CSV 파일 (output) 으로 추출
merged_df.to_csv(sampled_csv_path, index=False)
